In [ ]:
import asyncio
loop = asyncio.get_event_loop()

async def hello():
    print('Hello')
    await asyncio.sleep(3)
    print('World!')

if __name__ == '__main__':
    loop.run_until_complete(hello())

In [ ]:
from greenlet import sleep

def hello():
    sleep(3)
    print('Hello!')

if __name__ == '__main__':
    for i in range(10):
        hello()

In [ ]:
# do this for every iteration of the loop
await asyncio.sleep(0)

In [ ]:
# can't do:
socket.*
select.*
subprocess.*
os.waitpid.*
threading.*
multiprocessing.*
time.sleep

# use eventlet, gevent for monkey-patch

In [ ]:
import time
import threading

def calc_square(numbers):
    print("calculate square numbers")
    for n in numbers:
        time.sleep(0.)
        print('square:{}'.format(n*n))

def calc_cube(numbers):
    print("calculate cube of numbers")
    for n in numbers:
        time.sleep(0.)
        print('cube:{}'.format(n*n*n))

arr = [2,3,8,9,12,23,34,45]

t = time.time()

t1= threading.Thread(target=calc_square, args=(arr,))
t2= threading.Thread(target=calc_cube, args=(arr,))

t1.start()
t2.start()

t1.join()
t2.join()

print("done in : ",time.time()-t)
print("Hah... I am done with all my work now!")

In [ ]:
import time
import multiprocessing

def calc_square(numbers):
    for n in numbers:
        print('square ' + str(n*n))

def calc_hard(numbers):
    for n in numbers:
        print('cube ' + str(n*n*n*n*n*n))

if __name__ == "__main__":
    arr = [2,3,8,10,14,24,34]
    p1 = multiprocessing.Process(target=calc_square, args=(arr,))
    p2 = multiprocessing.Process(target=calc_hard, args=(arr,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("Done!")

In [ ]:
from threading import Thread, Event
from time import sleep

class MyThread(Thread):
    def __init__(self, event):
        Thread.__init__(self)
        self.stopped = event

    def run(self):
        while not self.stopped.wait(0.5):
            print("my thread")
            # call a function

stopFlag = Event()
thread = MyThread(stopFlag)
thread.start()
# this will stop the timer

stopFlag.set()

In [ ]:
from time import sleep
sleep(1)
print(1)

In [ ]:
from threading import Thread, Event
from time import sleep


class MyThread(Thread):
    def __init__(self, interval, thread_id):
        Thread.__init__(self)
        self._stop_event = Event()
        self.interval = interval
        self.thread_id = thread_id

    def run(self):
        while True:
            sleep(self.interval)
            print("my thread {}".format(self.thread_id))
            
    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
            
            
thread1 = MyThread(1, 1)
thread2 = MyThread(2, 2)
thread3 = MyThread(3, 3)


thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()

In [7]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name: ',__name__)
    if hasattr(os, 'getppid'):  # only available on Unix
        print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

main line
module name:  __main__
parent process: 22573
process id: 6763
function f
module name:  __main__
parent process: 6763
process id: 7357
hello bob


In [9]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 secs
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
4
9
16
25
36
49
64
81
400
8431
[8433, 8432, 8434, 8431]
We lacked patience and got a multiprocessing.TimeoutError
For the moment, the pool remains available for more work
Now the pool is closed and no longer available


In [4]:
%%time

from multiprocessing import Pool, TimeoutError
import time
import os 


def task(sleep_time, start_time):
    for i in range(10):
        print('pid: {} | sleep time: {} | time elapsed: {}'.format(os.getpid(), sleep_time, time.time()-start_time))
        time.sleep(sleep_time)

if __name__ == "__main__":
    
    results = []
    with Pool(processes=2) as pool:
        start_time = time.time()
        results = [pool.apply_async(task, args=(i,start_time)) for i in range(1,5)]
        
        for result in results:
            result.get()
    print("Done!")

pid: 11237 | sleep time: 2 | time elapsed: 0.0004024505615234375
pid: 11236 | sleep time: 1 | time elapsed: 0.0003819465637207031
pid: 11236 | sleep time: 1 | time elapsed: 1.0035035610198975
pid: 11236 | sleep time: 1 | time elapsed: 2.004843235015869
pid: 11237 | sleep time: 2 | time elapsed: 2.008927583694458
pid: 11236 | sleep time: 1 | time elapsed: 3.006819486618042
pid: 11236 | sleep time: 1 | time elapsed: 4.008209228515625
pid: 11237 | sleep time: 2 | time elapsed: 4.011738300323486
pid: 11236 | sleep time: 1 | time elapsed: 5.0096275806427
pid: 11236 | sleep time: 1 | time elapsed: 6.011520147323608
pid: 11237 | sleep time: 2 | time elapsed: 6.013641595840454
pid: 11236 | sleep time: 1 | time elapsed: 7.013319253921509
pid: 11236 | sleep time: 1 | time elapsed: 8.01515007019043
pid: 11237 | sleep time: 2 | time elapsed: 8.016420364379883
pid: 11236 | sleep time: 1 | time elapsed: 9.01710557937622
pid: 11237 | sleep time: 2 | time elapsed: 10.018184661865234
pid: 11236 | sleep

In [3]:
%%time

from multiprocessing import Pool, TimeoutError
import time
import os 


def task(sleep_time, start_time):
    for i in range(10):
        print('pid: {} | sleep time: {} | time elapsed: {}'.format(os.getpid(), sleep_time, time.time()-start_time))
        time.sleep(sleep_time)

if __name__ == "__main__":
    
    results = []
    with Pool(processes=2) as pool:
        start_time = time.time()
        results = [pool.apply_async(task, args=(i,start_time)) for i in range(1,5)]
        
        for result in results:
            result.get(timeout=1)
    print("Done!")

8

In [1]:
import time
import gevent
import random

def task(sleep_time, start_time):
    for i in range(10):
        print('task id: {} | elapased time: {}'.format(sleep_time, time.time()-start_time))
        gevent.sleep(sleep_time)
    

start_time = time.time()
threads = [gevent.spawn(task, i, start_time) for i in range(1,4)]
gevent.joinall(threads)

task id: 1 | elapased time: 0.00019240379333496094
task id: 2 | elapased time: 0.0002803802490234375
task id: 3 | elapased time: 0.0003216266632080078
task id: 1 | elapased time: 1.0003974437713623
task id: 2 | elapased time: 2.0005571842193604
task id: 1 | elapased time: 2.000673770904541
task id: 3 | elapased time: 3.0007517337799072
task id: 1 | elapased time: 3.000875949859619
task id: 2 | elapased time: 4.00097393989563
task id: 1 | elapased time: 4.001111268997192
task id: 1 | elapased time: 5.001620054244995
task id: 3 | elapased time: 6.00182580947876
task id: 2 | elapased time: 6.00198769569397
task id: 1 | elapased time: 6.002157211303711
task id: 1 | elapased time: 7.0026562213897705
task id: 2 | elapased time: 8.003229856491089
task id: 1 | elapased time: 8.003397226333618
task id: 3 | elapased time: 9.00221300125122
task id: 1 | elapased time: 9.003876209259033
task id: 2 | elapased time: 10.004219770431519
task id: 3 | elapased time: 12.003002166748047
task id: 2 | elapas